In [ ]:
# ✅ Route: Retrieve dataset
@data_routes.route("/session/<session_id>", methods=["GET"])
def get_data(session_id):
    """Retrieve dataset by session ID, ensuring all missing values are JSON-safe."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"].copy()  # ✅ Work with a COPY

    logging.info(f"📊 DataFrame types in session {session_id}:\n{df.dtypes}")

    # ✅ Get the correct data types for each column
    dtype_mapping = df.dtypes.apply(lambda x: str(x)).to_dict()

    # ✅ Convert datetime and timedelta columns to strings
    for col, dtype in dtype_mapping.items():
        if dtype.startswith("datetime64") or dtype.startswith("timedelta64"):
            df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S') if dtype.startswith("datetime64") else df[col].dt.total_seconds()

    # ✅ Convert category columns to strings
    for col, dtype in dtype_mapping.items():
        if dtype.startswith("category"):
            df[col] = df[col].astype(str)

    # ✅ Convert DataFrame to JSON-safe dictionary
    data_json_safe = df.to_dict(orient="records")

    # ✅ Replace NaN values with a special value
    for row in data_json_safe:
        for col, value in row.items():
            if pd.isna(value):
                row[col] = None

    return jsonify({
        "session_id": session_id,
        "name": session["name"],
        "data": data_json_safe,
        "columns": [
            {
                "field": col,
                "headerName": col,
                "dataType": str(dtype_mapping[col]),
            }
            for col in df.columns
        ],
        "total_rows": df.shape[0],
        "total_columns": df.shape[1]
    }), 200

In [ ]:
# ✅ Route: Retrieve dataset
@data_routes.route("/session/<session_id>", methods=["GET"])
def get_data(session_id):
    """Retrieve dataset by session ID, ensuring all missing values are JSON-safe."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"].copy()  # ✅ Work with a COPY

    logging.info(f"📊 DataFrame types in session {session_id}:\n{df.dtypes}")

    # ✅ Replace all NaN values (including in numeric columns)
    df = df.where(pd.notna(df), None)  # ✅ Converts NaN → None (JSON-safe)

    # ✅ Convert DataFrame to JSON-safe dictionary
    data_json_safe = []
    for row in df.itertuples(index=False, name=None):
        data_json_safe.append([None if pd.isna(x) else x for x in row])

    return jsonify({
        "session_id": session_id,
        "name": session["name"],
        "data": [dict(zip(df.columns, row)) for row in data_json_safe],  # ✅ Final Fix
        "columns": [
            {
                "field": col,
                "headerName": col,
                "dataType": str(df[col].dtype)  # ✅ Include column data type
            }
            for col in df.columns
        ],
        "total_rows": df.shape[0],
        "total_columns": df.shape[1]
    }), 200

In [ ]:
# ✅ API Routes for Dataset Management
from flask import Blueprint, request, jsonify
import pandas as pd
import logging
from models.data_manager import add_session, get_session, delete_session, get_available_sessions
import os 

# ✅ Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ✅ Define Blueprint for API routes
data_routes = Blueprint("data_routes", __name__)

# ✅ Route: Upload dataset
@data_routes.route("/upload", methods=["POST"])
def upload_data():
    """Upload a dataset and create a session."""
    try:
        if "file" not in request.files:
            logging.error("❌ No file provided in the request.")
            return jsonify({"error": "No file provided"}), 400

        file = request.files["file"]
        session_id = request.form.get("session_id", "default")

        # ✅ Automatically determine dataset name (remove .csv from dropdown name)
        name = request.form.get("name", file.filename)
        name = os.path.splitext(name)[0]  # Remove file extension

        # ✅ Try reading the file as CSV first, fallback to Excel
        try:
            df = pd.read_csv(file)
        except Exception:
            try:
                df = pd.read_excel(file, engine="openpyxl")
            except Exception as e:
                logging.error(f"❌ Unsupported or invalid file format: {file.filename} - {e}")
                return jsonify({"error": "Invalid or unsupported file format"}), 400

        # ✅ Add dataset session
        add_session(session_id, df, name)

        logging.info(f"✅ Dataset uploaded successfully under session {session_id} ({name})")
        return jsonify({"message": "File uploaded", "session_id": session_id, "name": name}), 200

    except Exception as e:
        logging.exception("❌ Error uploading dataset.")
        return jsonify({"error": str(e)}), 500



# ✅ Route: Retrieve dataset
@data_routes.route("/session/<session_id>", methods=["GET"])
def get_data(session_id):
    """Retrieve dataset by session ID, ensuring all missing values are JSON-safe."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"].copy()  # ✅ Work with a COPY
    logging.info(f"📊 DataFrame dtypes in session {session_id}:\n{df.dtypes}")

    # ✅ Preserve exact dtypes from Pandas
    dtype_mapping = {col: str(df[col].dtype) for col in df.columns}

    # ✅ Convert datetime and timedelta columns safely
    for col in df.select_dtypes(include=["datetime64[ns]"]):
        df[col] = df[col].apply(lambda x: x.isoformat() if pd.notna(x) else None)

    for col in df.select_dtypes(include=["timedelta64[ns]"]):
        df[col] = df[col].astype(str).replace("NaT", None)  # Keep timedelta readable

    # ✅ Ensure boolean columns stay as bool, even with missing values
    for col in df.select_dtypes(include=["bool"]):
        df[col] = df[col].astype(object)  # Keep True/False values

    # ✅ Convert missing values safely (NaN → None)
    json_data = df.where(pd.notna(df), None).to_dict(orient="records")

    return jsonify({
        "session_id": session_id,
        "name": session["name"],
        "data": json_data,  
        "columns": [
            {
                "field": col,
                "headerName": col,
                "dataType": dtype_mapping[col]  # ✅ Match Pandas dtype exactly
            }
            for col in df.columns
        ],
        "total_rows": df.shape[0],
        "total_columns": df.shape[1]
    }), 200











# ✅ Route: Get all available dataset sessions
@data_routes.route("/sessions", methods=["GET"])
def get_sessions():
    """Retrieve a list of all active dataset sessions."""
    return jsonify({"sessions": get_available_sessions()}), 200


# ✅ Route: Delete dataset
@data_routes.route("/delete/<session_id>", methods=["DELETE"])
def delete_data(session_id):
    """Delete a dataset session."""
    delete_session(session_id)
    return jsonify({"message": f"Session {session_id} deleted"}), 200

# ✅ Route: Remove duplicate rows from dataset
@data_routes.route("/remove_duplicates/<session_id>", methods=["GET", "POST"])
def remove_duplicates(session_id):
    """Remove duplicate rows from the dataset and return count."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"]  # ✅ Get the DataFrame
    original_size = len(df)
    df_cleaned = df.drop_duplicates(keep="first")  # ✅ Remove duplicates

    duplicates_removed = original_size - len(df_cleaned)

    # ✅ Update the session with the cleaned dataset
    session["df"] = df_cleaned

    return jsonify({
        "message": f"Removed {duplicates_removed} duplicate rows",
        "total_duplicates_removed": duplicates_removed,
        "new_total_rows": len(df_cleaned),
    }), 200


# ✅ Route: Update a specific cell in the dataset

@data_routes.route("/update_cell/<session_id>", methods=["POST"])
def update_cell(session_id):
    """Update a specific cell in the dataset and broadcast the change via WebSocket."""
    try:
        data = request.json
        column = data.get("column")
        row_index = data.get("row_index")
        new_value = data.get("new_value")

        # ✅ Retrieve the dataset session
        session = get_session(session_id)
        if session is None:
            return jsonify({"error": "Session not found"}), 404

        df = session["df"]  # ✅ Get the DataFrame

        # ✅ Ensure the column exists
        if column not in df.columns:
            return jsonify({"error": "Invalid column"}), 400

        # ✅ Ensure row index is within range
        if row_index < 0 or row_index >= len(df):
            return jsonify({"error": "Invalid row index"}), 400

        # ✅ Update the DataFrame
        df.at[row_index, column] = new_value

        # ✅ Broadcast the update to all WebSocket clients
        from websocket.socket_manager import socketio
        socketio.emit("update_data", {"session_id": session_id, "data": df.to_dict(orient="records")})

        return jsonify({"message": "Cell updated successfully"}), 200

    except Exception as e:
        logging.exception("❌ Error updating cell.")
        return jsonify({"error": str(e)}), 500
    
# ✅ Route: Detect duplicates

@data_routes.route("/detect_duplicates/<session_id>", methods=["GET"])
def detect_duplicates(session_id):
    """Detect duplicate rows in the dataset without modifying it."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"]  # ✅ Get the DataFrame

    # ✅ Identify duplicate rows (excluding the first occurrence)
    duplicate_mask = df.duplicated(keep=False)  # Marks all occurrences of duplicates

    # ✅ Count only the duplicates that would be removed
    duplicates_to_remove = df.duplicated(keep="first").sum()

    return jsonify({
        "message": f"✅ {duplicates_to_remove} duplicate rows detected.",
        "total_duplicates": int(duplicates_to_remove),  # Ensure integer type
    }), 200



@data_routes.route("/convert_datatype/<session_id>", methods=["POST"])
def convert_datatype(session_id):
    """Convert the data type of a specified column and persist it in the session."""
    try:
        data = request.json
        column = data.get("column")
        new_type = data.get("new_type")

        session = get_session(session_id)
        if session is None:
            logging.error(f"❌ Session {session_id} not found.")
            return jsonify({"error": "Session not found"}), 404

        df = session["df"].copy()  # ✅ Work with a COPY

        if column not in df.columns:
            logging.error(f"❌ Column {column} not found in session {session_id}.")
            return jsonify({"error": "Column not found"}), 400

        logging.info(f"🔄 Converting column '{column}' in session '{session_id}' to {new_type}.")

        # ✅ Force conversion explicitly
        try:
            if new_type == "int64":
                df[column] = df[column].astype("int64")
            elif new_type == "float64":
                df[column] = df[column].astype("float64")
            elif new_type == "object":  # Generic string conversion
                df[column] = df[column].astype("object")
            elif new_type == "boolean":
                df[column] = df[column].astype("boolean")
            elif new_type == "datetime64":
                df[column] = pd.to_datetime(df[column], errors="coerce")
            elif new_type == "timedelta64":
                df[column] = pd.to_timedelta(df[column], errors="coerce")
            elif new_type == "date":
                df[column] = pd.to_datetime(df[column], errors="coerce").dt.date
            elif new_type == "time":
                df[column] = pd.to_datetime(df[column], errors="coerce").dt.time
            elif new_type == "currency":
                currency_symbol = data.get("currency_symbol", "$")  # Get from request, default to "$"
                try:
                    df[column] = df[column].astype("float64").map(lambda x: f"{currency_symbol}{x:,.2f}" if pd.notna(x) else x)
                except ValueError as e:
                    logging.error(f"❌ Error formatting {column} as currency: {e}")
                    return jsonify({"error": f"Failed to convert {column} to currency"}), 400

            elif new_type == "percentage":
                df[column] = df[column].astype("float64").map(lambda x: f"{x * 100:.2f}%")  # ✅ Convert 0.85 → 85.00%
            elif new_type == "category":
                df[column] = df[column].astype("category")
            else:
                raise ValueError(f"Unsupported conversion type: {new_type}")

            # ✅ Overwrite the session explicitly
            add_session(session_id, df.copy(), session["name"])

            # ✅ Debug: Confirm session update
            session_after = get_session(session_id)
            logging.info(f"🔍 Session {session_id} after update:\n{session_after['df'].dtypes}")

            return jsonify({"message": f"Converted {column} to {new_type}"}), 200

        except Exception as e:
            logging.error(f"❌ Error converting {column} in session {session_id}: {e}")
            return jsonify({"error": str(e)}), 400

    except Exception as e:
        logging.error(f"❌ Unexpected error in convert_datatype: {e}")
        return jsonify({"error": str(e)}), 500




@data_routes.route("/replace_value/<session_id>", methods=["POST"])
def replace_value(session_id):
    """Replace a specific substring within a column in the dataset."""
    try:
        data = request.json
        column = data.get("column")
        find_value = data.get("find_value", "")
        replace_with = data.get("replace_with", "")

        session = get_session(session_id)
        if session is None:
            return jsonify({"error": "Session not found"}), 404

        df = session["df"].copy()  # Work with a COPY

        if column not in df.columns:
            return jsonify({"error": "Column not found"}), 400

        logging.info(f"🔄 Replacing '{find_value}' with '{replace_with}' in column '{column}' (Session: {session_id})")

        # ✅ Perform substring replacement for all values in the column
        df[column] = df[column].astype(str).str.replace(find_value, replace_with, regex=False)

        # ✅ Overwrite the session explicitly
        add_session(session_id, df.copy(), session["name"])

        return jsonify({
            "message": f"✅ Successfully replaced '{find_value}' with '{replace_with}' in column '{column}'",
            "updated_column": column
        }), 200

    except Exception as e:
        logging.error(f"❌ Error replacing value in session {session_id}: {e}")
        return jsonify({"error": str(e)}), 500

In [ ]:
import React, { useEffect, useState, useRef, useCallback } from "react";
import CustomHeader from "./CustomHeader";  // ✅ Import the custom header
import axios from "axios";
import { io } from "socket.io-client";
import { AgGridReact } from "ag-grid-react";
import "ag-grid-community/styles/ag-grid.css";
import "ag-grid-community/styles/ag-theme-alpine.css";
import { ModuleRegistry } from "ag-grid-enterprise";
import {
    ClientSideRowModelModule,
    MenuModule,
    IntegratedChartsModule,
    RangeSelectionModule,
    ColumnsToolPanelModule,
    FiltersToolPanelModule,
    ClipboardModule,
    ExcelExportModule,
    RowGroupingModule,
    SetFilterModule,
} from "ag-grid-enterprise";

// ✅ Register AgGrid modules
ModuleRegistry.registerModules([
    ClientSideRowModelModule,
    MenuModule,
    ColumnsToolPanelModule,
    FiltersToolPanelModule,
    ClipboardModule,
    IntegratedChartsModule,
    RangeSelectionModule,
    ExcelExportModule,
    RowGroupingModule,
    SetFilterModule,
]);

const API_URL = "http://127.0.0.1:5050";  // ✅ Backend API URL

function DataTable() {
    const [rowData, setRowData] = useState([]);  // ✅ Holds table data
    const [columnDefs, setColumnDefs] = useState([]);  // ✅ Holds column definitions
    const [sessions, setSessions] = useState([]);  // ✅ Holds available datasets
    const [selectedSession, setSelectedSession] = useState(null);  // ✅ Tracks selected dataset
    const [datasetShape, setDatasetShape] = useState("(0, 0)");  // ✅ Stores dataset shape from backend
    const gridRef = useRef();  // ✅ Reference to AgGrid instance
    const [showFormattingMenu, setShowFormattingMenu] = useState(false);
    const [showDuplicateOptions, setShowDuplicateOptions] = useState(false)
    const [keepDropdownOpen, setKeepDropdownOpen] = useState(false);
    const dropdownRef = useRef(null);
    const [message, setMessage] = useState(""); // ✅ Stores success message
    const [messageType, setMessageType] = useState(""); // ✅ Type: success or error
    const [filteredData, setFilteredData] = useState(null); // Stores original dataset when filtering
    const [showingDuplicates, setShowingDuplicates] = useState(false); // Tracks filter state
    const [showConvertTypeMenu, setShowConvertTypeMenu] = useState(false);
    const [showColumnSelectionMenu, setShowColumnSelectionMenu] = useState(false);
    const [columnSearch, setColumnSearch] = useState("");  // Search filter for column selection
    const [selectedColumnToConvert, setSelectedColumnToConvert] = useState("");  // Stores the selected column
    const [selectedDataType, setSelectedDataType] = useState("");  // Stores the new data type
    const columnMenuRef = useRef(null);
    const dataTypeMenuRef = useRef(null);
    const [selectedCurrencySymbol, setSelectedCurrencySymbol] = useState("$");

    const [showDataTypeMenu, setShowDataTypeMenu] = useState(false);  // Data type selection dropdown
    const [showReplaceMenu, setShowReplaceMenu] = useState(false);
    const [selectedColumnToReplace, setSelectedColumnToReplace] = useState("");
    const [replaceValue, setReplaceValue] = useState("");
    const [newReplaceValue, setNewReplaceValue] = useState("");





    



    // ✅ Fetch dataset from backend
    const fetchData = useCallback(async (sessionId) => {
        if (!sessionId) return;
        try {
            console.log(`📡 Fetching fresh data for session: ${sessionId}`);
    
            // ✅ Force a fresh request by adding a timestamp to the URL (prevents caching)
            const response = await axios.get(`${API_URL}/api/session/${sessionId}?_=${new Date().getTime()}`);
            
            console.log("🔄 Updated API Response:", response.data);
    
            if (response.data.data) {
                setRowData(response.data.data); // ✅ Update table data
                setColumnDefs(formatColumnDefs(response.data.columns)); // ✅ Update column types
    
                // ✅ Force update dataset shape
                setDatasetShape(`(${response.data.total_rows || 0}, ${response.data.total_columns || 0})`);
            } else {
                console.error("⚠️ No data found in API response.");
            }
        } catch (err) {
            console.error("❌ Error fetching updated data:", err);
        }
    }, []);
    

    // ✅ Fetch available sessions (datasets)
    const fetchSessions = useCallback(async () => {
        try {
            const response = await axios.get(`${API_URL}/api/get_sessions`);
            if (response.data.sessions) {
                const sessionEntries = Object.entries(response.data.sessions).map(([id, session]) => ({
                    id,
                    name: session.name || `Dataset ${id}`,
                }));
                setSessions(sessionEntries);

                // ✅ Auto-select the latest dataset if none is selected
                if (sessionEntries.length > 0 && !selectedSession) {
                    const latestSession = sessionEntries[sessionEntries.length - 1].id;
                    setSelectedSession(latestSession);
                    fetchData(latestSession);
                }
            }
        } catch (err) {
            console.error("❌ Error fetching sessions:", err);
        }
    }, [fetchData, selectedSession]);

    // ✅ Handle first-time data fetch
    useEffect(() => {
        fetchSessions();
    }, [fetchSessions]);

    // ✅ Handle dataset selection change
    useEffect(() => {
        if (selectedSession) {
            fetchData(selectedSession);
        }
    }, [selectedSession, fetchData]);

    // ✅ Real-time updates via WebSockets
    useEffect(() => {
        const socket = io(API_URL);
        socket.on("update_data", (newData) => {
            fetchSessions();
            if (newData.session_id === selectedSession) {
                setRowData(newData.data);
            }
        });
        return () => {
            socket.disconnect();
        };
    }, [selectedSession, fetchSessions]);

    




    // ✅ Format column definitions
    const formatColumnDefs = (columns = []) => {
        return columns.map((col) => ({
            field: col.field,
            headerName: col.headerName,
            editable: true,
            filter: "agSetColumnFilter",
            floatingFilter: true,
            resizable: true,
            sortable: true,
            enableValue: true,
            enableRowGroup: true,
            enablePivot: true,
            menuTabs: ["filterMenuTab", "columnsMenuTab"],
            suppressMenu: false,
            filterParams: {suppressMiniFilter: false, applyMiniFilterWhileTyping: true },
            // ✅ Ensure proper currency and percentage formatting
            valueFormatter: (params) => {
                if (!params.value) return params.value;

                if (col.dataType === "currency") {
                    return params.value; // The backend already formats currency (e.g., "$1,234.56")
                }
                if (col.dataType === "percentage") {
                    return params.value; // The backend already formats percentage (e.g., "85.00%")
                }
                if (col.dataType === "date") {
                    return new Date(params.value).toLocaleDateString("en-US"); // Converts to MM/DD/YYYY
                }
                if (col.dataType === "time") {
                    return new Date("1970-01-01 " + params.value).toLocaleTimeString("en-US", { hour12: false }); // Converts to HH:MM:SS
                }
                if (col.dataType === "datetime64") {
                    return new Date(params.value).toLocaleString("en-US"); // Converts to MM/DD/YYYY HH:MM:SS
                }
                if (col.dataType === "float64") {
                    return Number.isInteger(params.value) ? params.value.toFixed(1) : params.value;
                }
                return params.value;
            },
            

            // ✅ Dynamically set the data type
            dataType: col.dataType,  
    
            // ✅ Ensure the custom header receives the correct data type
            headerComponent: CustomHeader,
            headerComponentParams: {
                dataType: col.dataType || "Unknown",  // ✅ Pass dataType to custom header
            }
        }));
    };
    




    const detectDuplicates = async () => {
        if (!selectedSession) {
            console.error("❌ No dataset selected");
            return;
        }
    
        try {
            const response = await axios.get(`${API_URL}/api/detect_duplicates/${selectedSession}`);
            console.log("🔍 Duplicate Detection Response:", response.data);
    
            setMessage(response.data.message);
            setMessageType("success");
    
            // ✅ Close BOTH the sub-dropdown and main dropdown
            setShowFormattingMenu(false);
            setShowDuplicateOptions(false);
            setKeepDropdownOpen(false);
    
            setTimeout(() => setMessage(""), 5000);
        } catch (error) {
            console.error("❌ Error detecting duplicates:", error);
            setMessage("❌ Error detecting duplicates.");
            setMessageType("error");
    
            // ✅ Ensure the dropdown closes even if an error occurs
            setShowFormattingMenu(false);
            setShowDuplicateOptions(false);
            setKeepDropdownOpen(false);
    
            setTimeout(() => setMessage(""), 5000);
        }
    };






    // ✅ Function to Remove Duplicates
    const removeDuplicates = async () => {
        if (!selectedSession) {
            console.error("❌ No dataset selected");
            return;
        }
    
        try {
            const response = await axios.post(`${API_URL}/api/remove_duplicates/${selectedSession}`);
            console.log("🗑️ Remove Duplicates Response:", response.data);
    
            setMessage(response.data.message);
            setMessageType("success");
    
            // ✅ Refresh the dataset
            fetchData(selectedSession);
    
            // ✅ Close BOTH the sub-dropdown and main dropdown
            setShowFormattingMenu(false);
            setShowDuplicateOptions(false);
            setKeepDropdownOpen(false);
    
            setTimeout(() => setMessage(""), 5000);
        } catch (error) {
            console.error("❌ Error removing duplicates:", error);
            setMessage("❌ Failed to remove duplicates.");
            setMessageType("error");
    
            // ✅ Ensure the dropdown closes even if an error occurs
            setShowFormattingMenu(false);
            setShowDuplicateOptions(false);
            setKeepDropdownOpen(false);
    
            setTimeout(() => setMessage(""), 5000);
        }
    };




    const showOnlyDuplicates = () => {
        if (!rowData.length) {
            console.error("❌ No data available.");
            return;
        }
    
        // ✅ Count occurrences of each row
        const rowCounts = {};
        rowData.forEach(row => {
            const rowKey = JSON.stringify(row);
            rowCounts[rowKey] = (rowCounts[rowKey] || 0) + 1;
        });
    
        // ✅ Keep only duplicate rows
        const duplicatesOnly = rowData.filter(row => {
            const rowKey = JSON.stringify(row);
            return rowCounts[rowKey] > 1;
        });
    
        if (!duplicatesOnly.length) {
            // ✅ No duplicates found → Show message, but KEEP the table as is
            setMessage("🚫 No duplicate rows found.");
            setMessageType("warning");
    
            // ✅ Close dropdowns even if no duplicates are found
            setShowFormattingMenu(false);
            setShowDuplicateOptions(false);
            setKeepDropdownOpen(false);
    
            // ✅ Ensure the message disappears after 5 seconds
            setTimeout(() => setMessage(""), 5000);
            return; // ❌ Prevents toggling to "Restore All"
        }
    
        if (!showingDuplicates) {
            // ✅ Show only duplicate rows
            setFilteredData(rowData); // Store original data before filtering
            setRowData(duplicatesOnly);
            setMessage(`📌 Showing ${duplicatesOnly.length} duplicate rows.`);
            setShowingDuplicates(true);
        } else {
            // ✅ Restore original dataset
            setRowData(filteredData);
            setFilteredData(null);
            setMessage("✅ Restored all rows.");
            setShowingDuplicates(false);
        }
    
        setMessageType("success");
    
        // ✅ Close dropdowns after selecting "Show Duplicates"
        setShowFormattingMenu(false);
        setShowDuplicateOptions(false);
        setKeepDropdownOpen(false);
    
        // ✅ Hide message after 5 seconds
        setTimeout(() => setMessage(""), 5000);
    };
    
    




    useEffect(() => {
        const handleClickOutside = (event) => {
            if (
                dropdownRef.current && !dropdownRef.current.contains(event.target)
            ) {
                setShowFormattingMenu(false);  // ✅ Closes "Formatting" dropdown when clicking outside
                setShowDuplicateOptions(false);
                setShowConvertTypeMenu(false);
            }
        };
    
        document.addEventListener("mousedown", handleClickOutside);
        return () => {
            document.removeEventListener("mousedown", handleClickOutside);
        };
    }, []);
    
    
    
    
    
    


    // ✅ Handle dataset change
    const handleSessionChange = (event) => {
        const newSession = event.target.value;
        setSelectedSession(newSession);
    
        // ✅ Reset duplicate filtering state
        setFilteredData(null);
        setShowingDuplicates(false);
        setMessage(""); // ✅ Clear any messages
    };




    const convertColumnDataType = () => {
        if (!selectedColumnToConvert || !selectedDataType) {
            setMessage("⚠️ Please select a column and a target data type.");
            setMessageType("warning");
            setTimeout(() => setMessage(""), 4000);
            return;
        }
    
        console.log(`🟢 Converting ${selectedColumnToConvert} to ${selectedDataType}...`); // ✅ Debugging Step 1
    
        // ✅ Prepare request payload
        let requestData = { 
            column: selectedColumnToConvert, 
            new_type: selectedDataType 
        };
    
        // ✅ Handle currency conversion separately
        if (selectedDataType === "currency") {
            requestData.currency_symbol = selectedCurrencySymbol || "$";  // Default to "$" if not provided
        }
    
        axios.post(`${API_URL}/api/convert_datatype/${selectedSession}`, requestData, {
            headers: { "Content-Type": "application/json" }  
        })
        .then(response => {
            console.log("✅ Backend Response:", response.data); // ✅ Debugging Step 2
            setMessage(`✅ Column "${selectedColumnToConvert}" converted to ${selectedDataType}`);
            setMessageType("success");
    
            // ✅ Ensure frontend reloads updated dataset
            setTimeout(() => {
                console.log("🔄 Fetching updated dataset..."); // ✅ Debugging Step 3
                fetchData(selectedSession);
            }, 500);
    
            // ✅ Keep dropdown open to check if it's closing too early
            setKeepDropdownOpen(true);
        })
        .catch(error => {
            console.error("❌ Error converting column:", error);
            setMessage("❌ Failed to convert column.");
            setMessageType("error");
        });
    
        setTimeout(() => setMessage(""), 5000);
    };





    const replaceColumnValue = () => {
        if (!selectedColumnToReplace || replaceValue === undefined) {
            setMessage("⚠️ Please select a column and enter a value to replace.");
            setMessageType("warning");
            setTimeout(() => setMessage(""), 4000);
            return;
        }
    
        console.log(`🟢 Replacing "${replaceValue}" with "${newReplaceValue || ''}" in column "${selectedColumnToReplace}"...`);
    
        axios.post(`${API_URL}/api/replace_value/${selectedSession}`, { 
            column: selectedColumnToReplace, 
            find_value: replaceValue, 
            replace_with: newReplaceValue || ""  // If empty, it removes the value
        }, {
            headers: { "Content-Type": "application/json" }
        })
        .then(response => {
            console.log("✅ Backend Response:", response.data);
            setMessage(`✅ Replaced "${replaceValue}" with "${newReplaceValue || ''}" in ${selectedColumnToReplace}`);
            setMessageType("success");
    
            // ✅ Refresh data after replacement
            setTimeout(() => {
                console.log("🔄 Fetching updated dataset...");
                fetchData(selectedSession);
            }, 500);
    
            // ✅ Reset input fields
            setReplaceValue("");
            setNewReplaceValue("");
    
        })
        .catch(error => {
            console.error("❌ Error replacing value:", error);
            setMessage("❌ Failed to replace value.");
            setMessageType("error");
        });
    
        setTimeout(() => setMessage(""), 5000);
    };
    
    
    
    


    // ✅ Export functions
    const exportToCSV = () => gridRef.current.exportDataAsCsv();
    const exportToExcel = () => gridRef.current.exportDataAsExcel();

    return (
        <div className="ag-theme-alpine" style={{ height: "650px", width: "100%", padding: "15px", borderRadius: "8px", boxShadow: "0 4px 8px rgba(0,0,0,0.1)" }}>
            <h2>📊 Data Table</h2>
            <div style={{ display: "flex", justifyContent: "space-between", alignItems: "center", marginBottom: "10px" }}>
                
                {/* ✅ Dataset Selection & Shape Display */}
                <div style={{ display: "flex", alignItems: "center", gap: "15px" }}>
                    <div>
                        <label>Select Dataset: </label>
                        <select onChange={handleSessionChange} value={selectedSession}>
                            {sessions.map((session) => (
                                <option key={session.id} value={session.id}>{session.name}</option>
                            ))}
                        </select>
                    </div>
    
                    {/* ✅ Dataset Shape Display Box (Backend-controlled) */}
                    <div 
                        style={{ 
                            padding: "5px 10px",
                            border: "1px solid #ccc",
                            borderRadius: "5px",
                            background: "#f9f9f9",
                            fontSize: "14px",
                            fontWeight: "bold"
                        }}
                    >
                        {datasetShape}
                    </div>
                </div>
    
                {/* ✅ Formatting Button */}
                <div style={{ position: "relative", marginLeft: "20px" }} ref={dropdownRef}>
                    {/* ⚙️ Main Formatting Button */}
                    <button 
                        onClick={() => setShowFormattingMenu(prev => !prev)} 
                        className="formatting-button"
                    >
                        ⚙️ Formatting ▼
                    </button>

                    {/* ✅ Main Dropdown Menu */}
                    {showFormattingMenu && (
                        <div 
                            className="dropdown-menu"
                            onMouseEnter={() => setKeepDropdownOpen(true)}
                            onMouseLeave={() => setKeepDropdownOpen(false)}
                        >
                            {/* 🔍 Duplicates Submenu */}
                            <div 
                                className="dropdown-item"
                                onMouseEnter={() => setShowDuplicateOptions(true)}
                                onMouseLeave={() => setShowDuplicateOptions(false)}
                            >
                                🔍 Duplicates &rsaquo;

                                {/* ✅ Duplicates Submenu Options */}
                                {showDuplicateOptions && (
                                    <div 
                                        className="submenu"
                                        onMouseEnter={() => setShowDuplicateOptions(true)}
                                        onMouseLeave={() => setShowDuplicateOptions(false)}
                                    >
                                        <button className="submenu-item" onClick={detectDuplicates}>
                                            🔍 Detect Duplicates
                                        </button>
                                        <button className={`submenu-item ${showingDuplicates ? "restore-btn" : ""}`} onClick={showOnlyDuplicates}>
                                            {showingDuplicates ? "🔄 Restore All" : "📌 Show Duplicates"}
                                        </button>
                                        <button className="submenu-item red" onClick={removeDuplicates}>
                                            ❌ Remove Duplicates
                                        </button>
                                    </div>
                                )}
                            </div>

                            {/* 🔄 Convert Data Type Submenu */}
                            <div 
                                className="dropdown-item"
                                onMouseEnter={() => setShowConvertTypeMenu(true)}
                                onMouseLeave={() => setShowConvertTypeMenu(false)}
                            >
                                🔄 Convert Data Type &rsaquo;

                                {/* ✅ Column Selection Submenu */}
                                {showConvertTypeMenu && (
                                    <div 
                                        className="submenu wider-submenu" 
                                        onMouseEnter={() => setShowConvertTypeMenu(true)}
                                        onMouseLeave={() => setShowConvertTypeMenu(false)}
                                    >
                                        <label className="submenu-label">Select Column:</label>
                                        <input
                                            type="text"
                                            placeholder="Search column..."
                                            className="search-box"
                                            value={columnSearch}
                                            onChange={(e) => setColumnSearch(e.target.value)}
                                        />
                                        <select 
                                            className="column-select-box"
                                            value={selectedColumnToConvert || ""}
                                            onChange={(e) => setSelectedColumnToConvert(e.target.value)}
                                        >
                                            <option value="" disabled>Select a column</option>
                                            {columnDefs
                                                .filter(col => col.headerName.toLowerCase().includes(columnSearch.toLowerCase()))
                                                .map((col) => (
                                                    <option key={col.field} value={col.field}>
                                                        {col.headerName}
                                                    </option>
                                                ))
                                            }
                                        </select>

                                        {/* ✅ Show Data Type Selection Only If Column is Selected */}
                                        {selectedColumnToConvert && (
                                            <div className="submenu" style={{ marginTop: "10px" }}>
                                                <label className="submenu-label">Convert To:</label>
                                                <select 
                                                    className="datatype-select-box"
                                                    value={selectedDataType}
                                                    onChange={(e) => setSelectedDataType(e.target.value)}
                                                >
                                                    <option value="" disabled>Select Data Type</option>
                                                    <option value="int64">Integer</option>
                                                    <option value="float64">Float</option>
                                                    <option value="object">String</option>
                                                    <option value="boolean">Boolean</option>
                                                    <option value="datetime64">Datetime</option>
                                                    <option value="timedelta64">Timedelta</option>
                                                    <option value="date">Date</option>
                                                    <option value="time">Time</option>
                                                    <option value="currency">Currency</option>
                                                    <option value="percentage">Percentage</option>
                                                    <option value="category">Category</option>
                                                </select>

                                                {/* ✅ Show Currency Selection Only If "Currency" is Selected */}
                                                {selectedDataType === "currency" && (
                                                    <div className="submenu" style={{ marginTop: "10px" }}>
                                                        <label className="submenu-label">Select Currency:</label>
                                                        <select 
                                                            className="currency-select-box"
                                                            value={selectedCurrencySymbol}
                                                            onChange={(e) => setSelectedCurrencySymbol(e.target.value)}
                                                        >
                                                            <option value="$">USD ($)</option>
                                                            <option value="€">Euro (€)</option>
                                                            <option value="£">Pound (£)</option>
                                                            <option value="¥">Yen (¥)</option>
                                                        </select>
                                                    </div>
                                                )}

                                                {/* ✅ Apply Conversion Button */}
                                                <button 
                                                    className="apply-conversion-btn"
                                                    onClick={convertColumnDataType}
                                                >
                                                    ✅ Apply
                                                </button>
                                            </div>
                                        )}
                                    </div>
                                )}
                            </div>

                            {/* 🆕 🔄 Replace With Submenu */}
                            <div 
                                className="dropdown-item"
                                onMouseEnter={() => setShowReplaceMenu(true)}
                                onMouseLeave={() => setShowReplaceMenu(false)}
                            >
                                🔄 Replace With &rsaquo;

                                {/* ✅ Replace Submenu */}
                                {showReplaceMenu && (
                                    <div 
                                        className="submenu wider-submenu" 
                                        onMouseEnter={() => setShowReplaceMenu(true)}
                                        onMouseLeave={() => setShowReplaceMenu(false)}
                                    >
                                        <label className="submenu-label">Select Column:</label>
                                        <input
                                            type="text"
                                            placeholder="Search column..."
                                            className="search-box"
                                            value={columnSearch}
                                            onChange={(e) => setColumnSearch(e.target.value)}
                                        />
                                        <select 
                                            className="column-select-box"
                                            value={selectedColumnToReplace || ""}
                                            onChange={(e) => setSelectedColumnToReplace(e.target.value)}
                                        >
                                            <option value="" disabled>Select a column</option>
                                            {columnDefs
                                                .filter(col => col.headerName.toLowerCase().includes(columnSearch.toLowerCase()))
                                                .map((col) => (
                                                    <option key={col.field} value={col.field}>
                                                        {col.headerName}
                                                    </option>
                                                ))
                                            }
                                        </select>

                                        {/* ✅ Replacement Inputs */}
                                        {selectedColumnToReplace && (
                                            <div className="submenu" style={{ marginTop: "10px" }}>
                                                <label className="submenu-label">Find:</label>
                                                <input 
                                                    type="text" 
                                                    placeholder="Enter value to replace" 
                                                    className="replace-input"
                                                    value={replaceValue}
                                                    onChange={(e) => setReplaceValue(e.target.value)}
                                                />

                                                <label className="submenu-label">Replace With:</label>
                                                <input 
                                                    type="text" 
                                                    placeholder="Enter new value (leave empty to remove)" 
                                                    className="replace-input"
                                                    value={newReplaceValue}
                                                    onChange={(e) => setNewReplaceValue(e.target.value)}
                                                />

                                                {/* ✅ Apply Replacement Button */}
                                                <button 
                                                    className="apply-replacement-btn"
                                                    onClick={replaceColumnValue}
                                                >
                                                    ✅ Apply
                                                </button>
                                            </div>
                                        )}
                                    </div>
                                )}
                            </div>
                        </div>
                    )}
                </div>


                {/* ✅ Export Buttons */}
                <div>
                    <button onClick={exportToCSV} style={{ marginRight: "10px" }}>Export CSV</button>
                    <button onClick={exportToExcel}>Export Excel</button>
                </div>
            </div>

            {/* ✅ Success/Error Message Box */}
            {message && (
                <div 
                    style={{
                        padding: "10px",
                        marginBottom: "10px",
                        borderRadius: "5px",
                        textAlign: "center",
                        fontSize: "16px",
                        fontWeight: "bold",
                        backgroundColor: messageType === "success" ? "#d4edda" : "#f8d7da",
                        color: messageType === "success" ? "#155724" : "#721c24",
                        border: messageType === "success" ? "1px solid #c3e6cb" : "1px solid #f5c6cb",
                    }}
                >
                    {message}
                </div>
            )}

    
            {/* ✅ AgGrid Table */}
            <AgGridReact
                ref={gridRef}
                rowData={rowData}
                columnDefs={columnDefs}
                pagination={true}
                paginationPageSize={10}
                animateRows={true}
                rowSelection="multiple"
                suppressMenuHide={true}
                suppressHorizontalScroll={false}
                enableRangeSelection={true}
                enableClipboard={true}
                singleClickEdit={true}  // ✅ Enable single-click editing
                stopEditingWhenCellsLoseFocus={true}  // ✅ Save changes automatically
                autoGroupColumnDef={{
                    headerName: "Group",
                    field: "group",
                    cellRenderer: "agGroupCellRenderer",
                    cellRendererParams: {
                        checkbox: true
                    }
                }}

                
                
                sideBar={{
                    toolPanels: [
                        { id: "columns", labelDefault: "Columns", toolPanel: "agColumnsToolPanel", minWidth: 300 },
                        { id: "filters", labelDefault: "Filters", toolPanel: "agFiltersToolPanel", minWidth: 300 },
                    ],
                    defaultToolPanel: "columns",
                }}
                rowGroupPanelShow="always"
                pivotPanelShow="always"
                groupDisplayType="groupRows"
                


                defaultColDef={{
                    sortable: true,
                    resizable: true,
                    editable: true,
                    floatingFilter: true,
                    filter: "agSetColumnFilter",
                    enableValue: true,
                    enableRowGroup: true,
                    enablePivot: true,
                   
                }}

            />
        </div>
    );
    
}

export default DataTable;

In [ ]:
# ✅ Manages dataset sessions and handles server startup
import os
import time
import threading
import pandas as pd
import logging

# ✅ Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ✅ In-memory storage for dataset sessions
sessions = {}


def add_session(session_id, data, name="Untitled Dataset"):
    """Add a new dataset session."""
    if session_id in sessions:
        logging.warning(f"⚠️ Session {session_id} already exists. Overwriting data.")
    
    # ✅ Store session with metadata
    sessions[session_id] = {
        "df": data,
        "name": name,
        "created_at": time.time(),
    }
    logging.info(f"✅ Session {session_id} added ({name}).")



def get_session(session_id):
    """Retrieve dataset session by ID."""
    session = sessions.get(session_id)
    if session is None:
        logging.error(f"❌ Session {session_id} not found.")
        return None  # No JSON response here, handled at API level
    return session


def get_available_sessions():
    """Return a dictionary of active session IDs and metadata."""
    return {sid: {"name": s["name"], "created_at": s["created_at"]} for sid, s in sessions.items()}


def delete_session(session_id):
    """Delete a dataset session."""
    if session_id in sessions:
        del sessions[session_id]
        logging.info(f"🗑️ Session {session_id} deleted.")
    else:
        logging.warning(f"⚠️ Attempted to delete non-existent session {session_id}.")


def start_server():
    """Initialize backend services."""
    logging.info("🔥 Initializing dataset session manager...")
    os.makedirs("datasets", exist_ok=True)  # ✅ Ensure dataset storage exists
    logging.info("✅ Dataset directory verified.")

    # ✅ Start background cleanup process
    threading.Thread(target=session_cleanup, daemon=True).start()


def session_cleanup():
    """Background task to clean up inactive sessions."""
    logging.info("🔄 Session cleanup process started.")
    while True:
        time.sleep(30)  # ✅ Cleanup every 30 seconds (prevents CPU overload)
        inactive_sessions = [sid for sid, data in sessions.items() if data["df"].empty]
        for sid in inactive_sessions:
            delete_session(sid)


In [ ]:
# ✅ API Routes for Dataset Management
from flask import Blueprint, request, jsonify
import pandas as pd
import logging
from models.data_manager import add_session, get_session, delete_session, get_available_sessions
import os 

# ✅ Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ✅ Define Blueprint for API routes
data_routes = Blueprint("data_routes", __name__)

# ✅ Route: Upload dataset
import pickle
import io

# ✅ Route: Upload dataset
@data_routes.route("/upload", methods=["POST"])
def upload_data():
    """Upload a dataset and create a session."""
    try:
        if "file" not in request.files:
            logging.error("❌ No file provided in the request.")
            return jsonify({"error": "No file provided"}), 400

        file = request.files["file"]
        session_id = request.form.get("session_id", "default")

        # ✅ Automatically determine dataset name (remove `.pkl`)
        name = request.form.get("name", file.filename)
        name = os.path.splitext(name)[0]  # Remove file extension

        try:
            # ✅ Read Pickle file
            buffer = io.BytesIO(file.read())  # Read file as binary
            df = pickle.load(buffer)  # Deserialize Pickle to DataFrame

            # ✅ Ensure it's a valid DataFrame
            if not isinstance(df, pd.DataFrame):
                raise ValueError("Uploaded file does not contain a valid DataFrame.")

        except Exception as e:
            logging.error(f"❌ Failed to load Pickle file: {file.filename} - {e}")
            return jsonify({"error": "Invalid or corrupted file format"}), 400

        # ✅ Add dataset session
        add_session(session_id, df, name)

        logging.info(f"✅ Dataset uploaded successfully under session {session_id} ({name})")
        return jsonify({"message": "File uploaded", "session_id": session_id, "name": name}), 200

    except Exception as e:
        logging.exception("❌ Error uploading dataset.")
        return jsonify({"error": str(e)}), 500



# ✅ Route: Retrieve dataset
@data_routes.route("/session/<session_id>", methods=["GET"])
def get_data(session_id):
    """Retrieve dataset by session ID, ensuring all missing values are JSON-safe."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"].copy()  # ✅ Work with a COPY

    logging.info(f"📊 DataFrame types in session {session_id}:\n{df.dtypes}")

    # ✅ Get the correct data types for each column
    dtype_mapping = df.dtypes.apply(lambda x: str(x)).to_dict()

    # ✅ Convert DataFrame to JSON-safe dictionary
    data_json_safe = df.to_dict(orient="records")

    # ✅ Replace NaN values with a special value
    for row in data_json_safe:
        for col, value in row.items():
            if pd.isna(value):
                row[col] = None

    return jsonify({
        "session_id": session_id,
        "name": session["name"],
        "data": data_json_safe,
        "columns": [
            {
                "field": col,
                "headerName": col,
                "dataType": dtype_mapping[col],
            }
            for col in df.columns
        ],
        "total_rows": df.shape[0],
        "total_columns": df.shape[1]
    }), 200











# ✅ Route: Get all available dataset sessions
@data_routes.route("/sessions", methods=["GET"])
def get_sessions():
    """Retrieve a list of all active dataset sessions."""
    return jsonify({"sessions": get_available_sessions()}), 200


# ✅ Route: Delete dataset
@data_routes.route("/delete/<session_id>", methods=["DELETE"])
def delete_data(session_id):
    """Delete a dataset session."""
    delete_session(session_id)
    return jsonify({"message": f"Session {session_id} deleted"}), 200

# ✅ Route: Remove duplicate rows from dataset
@data_routes.route("/remove_duplicates/<session_id>", methods=["GET", "POST"])
def remove_duplicates(session_id):
    """Remove duplicate rows from the dataset and return count."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"]  # ✅ Get the DataFrame
    original_size = len(df)
    df_cleaned = df.drop_duplicates(keep="first")  # ✅ Remove duplicates

    duplicates_removed = original_size - len(df_cleaned)

    # ✅ Update the session with the cleaned dataset
    session["df"] = df_cleaned

    return jsonify({
        "message": f"Removed {duplicates_removed} duplicate rows",
        "total_duplicates_removed": duplicates_removed,
        "new_total_rows": len(df_cleaned),
    }), 200


# ✅ Route: Update a specific cell in the dataset

@data_routes.route("/update_cell/<session_id>", methods=["POST"])
def update_cell(session_id):
    """Update a specific cell in the dataset and broadcast the change via WebSocket."""
    try:
        data = request.json
        column = data.get("column")
        row_index = data.get("row_index")
        new_value = data.get("new_value")

        # ✅ Retrieve the dataset session
        session = get_session(session_id)
        if session is None:
            return jsonify({"error": "Session not found"}), 404

        df = session["df"]  # ✅ Get the DataFrame

        # ✅ Ensure the column exists
        if column not in df.columns:
            return jsonify({"error": "Invalid column"}), 400

        # ✅ Ensure row index is within range
        if row_index < 0 or row_index >= len(df):
            return jsonify({"error": "Invalid row index"}), 400

        # ✅ Update the DataFrame
        df.at[row_index, column] = new_value

        # ✅ Broadcast the update to all WebSocket clients
        from websocket.socket_manager import socketio
        socketio.emit("update_data", {"session_id": session_id, "data": df.to_dict(orient="records")})

        return jsonify({"message": "Cell updated successfully"}), 200

    except Exception as e:
        logging.exception("❌ Error updating cell.")
        return jsonify({"error": str(e)}), 500
    
# ✅ Route: Detect duplicates

@data_routes.route("/detect_duplicates/<session_id>", methods=["GET"])
def detect_duplicates(session_id):
    """Detect duplicate rows in the dataset without modifying it."""
    session = get_session(session_id)
    if session is None:
        return jsonify({"error": "Session not found"}), 404

    df = session["df"]  # ✅ Get the DataFrame

    # ✅ Identify duplicate rows (excluding the first occurrence)
    duplicate_mask = df.duplicated(keep=False)  # Marks all occurrences of duplicates

    # ✅ Count only the duplicates that would be removed
    duplicates_to_remove = df.duplicated(keep="first").sum()

    return jsonify({
        "message": f"✅ {duplicates_to_remove} duplicate rows detected.",
        "total_duplicates": int(duplicates_to_remove),  # Ensure integer type
    }), 200



@data_routes.route("/convert_datatype/<session_id>", methods=["POST"])
def convert_datatype(session_id):
    """Convert the data type of a specified column and persist it in the session."""
    try:
        data = request.json
        column = data.get("column")
        new_type = data.get("new_type")

        session = get_session(session_id)
        if session is None:
            logging.error(f"❌ Session {session_id} not found.")
            return jsonify({"error": "Session not found"}), 404

        df = session["df"].copy()  # ✅ Work with a COPY

        if column not in df.columns:
            logging.error(f"❌ Column {column} not found in session {session_id}.")
            return jsonify({"error": "Column not found"}), 400

        logging.info(f"🔄 Converting column '{column}' in session '{session_id}' to {new_type}.")

        # ✅ Force conversion explicitly
        try:
            if new_type == "int64":
                df[column] = df[column].astype("int64")
            elif new_type == "float64":
                df[column] = df[column].astype("float64")
            elif new_type == "object":  # Generic string conversion
                df[column] = df[column].astype("object")
            elif new_type == "boolean":
                df[column] = df[column].astype("boolean")
            elif new_type == "datetime64":
                df[column] = pd.to_datetime(df[column], errors="coerce")
            elif new_type == "timedelta64":
                df[column] = pd.to_timedelta(df[column], errors="coerce")
            elif new_type == "date":
                df[column] = pd.to_datetime(df[column], errors="coerce").dt.date
            elif new_type == "time":
                df[column] = pd.to_datetime(df[column], errors="coerce").dt.time
            elif new_type == "currency":
                currency_symbol = data.get("currency_symbol", "$")  # Get from request, default to "$"
                try:
                    df[column] = df[column].astype("float64").map(lambda x: f"{currency_symbol}{x:,.2f}" if pd.notna(x) else x)
                except ValueError as e:
                    logging.error(f"❌ Error formatting {column} as currency: {e}")
                    return jsonify({"error": f"Failed to convert {column} to currency"}), 400

            elif new_type == "percentage":
                df[column] = df[column].astype("float64").map(lambda x: f"{x * 100:.2f}%")  # ✅ Convert 0.85 → 85.00%
            elif new_type == "category":
                df[column] = df[column].astype("category")
            else:
                raise ValueError(f"Unsupported conversion type: {new_type}")

            # ✅ Overwrite the session explicitly
            add_session(session_id, df.copy(), session["name"])

            # ✅ Debug: Confirm session update
            session_after = get_session(session_id)
            logging.info(f"🔍 Session {session_id} after update:\n{session_after['df'].dtypes}")

            return jsonify({"message": f"Converted {column} to {new_type}"}), 200

        except Exception as e:
            logging.error(f"❌ Error converting {column} in session {session_id}: {e}")
            return jsonify({"error": str(e)}), 400

    except Exception as e:
        logging.error(f"❌ Unexpected error in convert_datatype: {e}")
        return jsonify({"error": str(e)}), 500




@data_routes.route("/replace_value/<session_id>", methods=["POST"])
def replace_value(session_id):
    """Replace a specific substring within a column in the dataset."""
    try:
        data = request.json
        column = data.get("column")
        find_value = data.get("find_value", "")
        replace_with = data.get("replace_with", "")

        session = get_session(session_id)
        if session is None:
            return jsonify({"error": "Session not found"}), 404

        df = session["df"].copy()  # Work with a COPY

        if column not in df.columns:
            return jsonify({"error": "Column not found"}), 400

        logging.info(f"🔄 Replacing '{find_value}' with '{replace_with}' in column '{column}' (Session: {session_id})")

        # ✅ Perform substring replacement for all values in the column
        df[column] = df[column].astype(str).str.replace(find_value, replace_with, regex=False)

        # ✅ Overwrite the session explicitly
        add_session(session_id, df.copy(), session["name"])

        return jsonify({
            "message": f"✅ Successfully replaced '{find_value}' with '{replace_with}' in column '{column}'",
            "updated_column": column
        }), 200

    except Exception as e:
        logging.error(f"❌ Error replacing value in session {session_id}: {e}")
        return jsonify({"error": str(e)}), 500

# notebook_integration.py

In [ ]:
import requests
import pandas as pd
import os
import sys
from IPython.core.display import display, HTML

# Define Backend API URL
API_URL = "http://127.0.0.1:5050"

def show(df=None, name=None, session_id=None):
    """Display the UI inside Jupyter Notebook with dataset switching support."""

    if df is not None and not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a Pandas DataFrame.")

    if df is not None:
        # ✅ Automatically infer variable name if `name` is not provided
        if name is None:
            import inspect
            frame = inspect.currentframe().f_back
            name = [var_name for var_name, var_val in frame.f_locals.items() if var_val is df]
            name = name[0] if name else "Untitled_Dataset"  # Default to "Untitled_Dataset" if name detection fails

        # ✅ Ensure a clean dataset name (no `.csv`)
        filename = name  # ✅ Keep dataset name clean

        # ✅ Fix Upload Issue: Encode CSV Properly
        files = {"file": (filename, df.to_csv(index=False).encode('utf-8'), "text/csv")}
        response = requests.post(f"{API_URL}/api/upload", files=files, data={"session_id": name})

        if response.status_code != 200:
            try:
                error_message = response.json().get('error', 'Unknown error')
            except:
                error_message = response.text  # Handle case where response is not JSON
            raise ValueError(f"Failed to load data: {error_message}")

        session_id = response.json()["session_id"]

    elif session_id:
        # ✅ Validate if session exists
        response = requests.get(f"{API_URL}/api/session/{session_id}")
        if response.status_code != 200:
            raise ValueError(f"Invalid session_id: {session_id}")

    else:
        # ✅ Get latest session if no session_id is provided
        response = requests.get(f"{API_URL}/api/get_sessions")
        sessions = response.json().get("sessions", {})
        if sessions:
            session_id = list(sessions.keys())[-1]  # Get the latest session
        else:
            raise ValueError("No active sessions available. Please upload a dataset first.")

    # ✅ Define Web UI URL
    server_url = f"{API_URL}/?session_id={session_id}"

    # ✅ Display the UI inside Jupyter Notebook
    iframe_html = f"""
    <iframe src="{server_url}" width="100%" height="600px" style="border:none;"></iframe>
    <p style="margin-top:10px;">
        <a href="{server_url}" target="_blank" style="font-size:14px; text-decoration:none; color:#007bff;">
            🔗 Open in Web Browser
        </a>
    </p>
    """
    display(HTML(iframe_html))
